In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split, KFold
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler



In [6]:
train = pd.read_csv('/Users/rfalcao/Documents/train 2.csv')


In [7]:
y_train = train["label"]  # Copy labels
y_train= to_categorical(y_train, num_classes = 10) # One-hot encode the labels
train = train.drop(columns=["label"]) / 255  # Normalize without modifying the original
train_xCNN=train.to_numpy().reshape(-1, 28, 28, 1)

In [11]:
# model_CNN = keras.Sequential([
#     keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
#     keras.layers.MaxPooling2D((2,2)),
#     keras.layers.Conv2D(64, (3,3), activation='relu'),
#     keras.layers.MaxPooling2D((2,2)),
#     keras.layers.Flatten(),
#     keras.layers.Dense(128, activation='relu'),
#     keras.layers.Dense(10, activation='softmax')  # 10 output classes
# ])
model_CNN = keras.Sequential([
    keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28,28,1)),
    keras.layers.BatchNormalization(),
    keras.layers.MaxPooling2D((2,2)),
    
    keras.layers.Conv2D(64, (3,3), activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(64, (3,3), strides=2, padding='same', activation='relu'),  # Strided convolution instead of pooling
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.4),  # Regularization
    
    keras.layers.Flatten(),
    keras.layers.Dense(128),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('relu'),
    keras.layers.Dropout(0.4),  # Regularization
    keras.layers.Dense(10, activation='softmax')  # 10 output classes
])

model_CNN.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Summary
model_CNN.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 26, 26, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 11, 11, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 11, 11, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 6, 6, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 6, 6, 64)       │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 353,226 (1.35 MB)

 Trainable params: 352,650 (1.35 MB)

 Non-trainable params: 576 (2.25 KB)

In [12]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.10,  
        width_shift_range=0.1, 
        height_shift_range=0.1)

In [13]:
epochs = 45
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.98 ** x)

# Train-test split
X_train2, X_val2, Y_train2, Y_val2 = train_test_split(train_xCNN, y_train, test_size=0.1)  # FIXED

# Train model
history = model_CNN.fit(datagen.flow(X_train2, Y_train2, batch_size=64),  # FIXED
                        epochs=epochs,
                        steps_per_epoch=len(X_train2) // 64,
                        validation_data=(X_val2, Y_val2),
                        callbacks=[annealer],
                        verbose=1)

# Evaluate model
val_loss, val_acc = model_CNN.evaluate(X_val2, Y_val2)
print(f"Validation Accuracy: {val_acc:.4f}")

Epoch 1/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 34s 50ms/step - accuracy: 0.7663 - loss: 0.7312 - val_accuracy: 0.9733 - val_loss: 0.0951 - learning_rate: 0.0010
Epoch 2/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9062 - loss: 0.2506 - val_accuracy: 0.9724 - val_loss: 0.0967 - learning_rate: 9.8000e-04
Epoch 3/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 30s 52ms/step - accuracy: 0.9574 - loss: 0.1443 - val_accuracy: 0.9862 - val_loss: 0.0460 - learning_rate: 9.6040e-04
Epoch 4/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9219 - loss: 0.1469 - val_accuracy: 0.9862 - val_loss: 0.0454 - learning_rate: 9.4119e-04
Epoch 5/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 28s 48ms/step - accuracy: 0.9680 - loss: 0.1037 - val_accuracy: 0.9743 - val_loss: 0.0856 - learning_rate: 9.2237e-04
Epoch 6/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9688 - loss: 0.0733 - val_accuracy: 0.9743 - val_loss: 0.0844 - learning_rate: 9.0392e-04
Epoch 7/45
590/590 ━━━━━━━━━━━━━━━━━━━━ 28s 47ms/step - accura

In [15]:
model_CNN.save('trained_model.h5')
model_CNN.save('trained_model.keras')

In [31]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import string

# Load trained model
model = load_model('trained_model.h5')

def segment_digits_with_labels(image_path):
    image = cv2.imread(image_path)  # Read in color (BGR) to allow colored labels
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)  # Convert to grayscale for processing
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    digits = []
    labels = string.ascii_uppercase  # Excel column labels (A, B, C, D, ...)
    label_index = 0  # Start with 'A'

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if w > 10 and h > 10:
            digit_image = gray[y:y+h, x:x+w]
            digit_image_resized = cv2.resize(digit_image, (28, 28))
            digit_image_normalized = digit_image_resized / 255.0
            digit_image_normalized = digit_image_normalized.reshape(1, 28, 28, 1)

            prediction = model.predict(digit_image_normalized)
            predicted_digit = np.argmax(prediction)

            # Label the bounding box with Excel column labels
            label = labels[label_index] if label_index < len(labels) else 'Z'
            digits.append((predicted_digit, label, (x, y, w, h)))

            # Draw the bounding box
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # Draw label above the box
            cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

            # Draw predicted digit below the box
            cv2.putText(image, str(predicted_digit), (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            label_index += 1  # Move to the next label

    return image, digits




In [33]:
image_path = '/Users/rfalcao/Documents/FYP/Digit_identifier/TestSeq2.jpg'

segmented_image, predicted_digits = segment_digits_with_labels(image_path)

# Display the result
plt.imshow(segmented_image, cmap='gray')
plt.axis('off')
plt.show()

# Print predicted digits with their Excel-like labels
for digit, label, position in predicted_digits:
    print(f"Predicted Digit: {digit} at label {label} with position {position}")

[ WARN:0@18563.985] global loadsave.cpp:268 findDecoder imread_('/Users/rfalcao/Documents/FYP/Digit_identifier/TestSeq2.jpg'): can't open/read file: check file path/integrity


error: OpenCV(4.11.0) /Users/xperience/GHA-Actions-OpenCV/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:199: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'
